In [272]:
data = open("data.txt", "r").read()

In [273]:
# Sensor (x, y), Beacon (x, y)
sensors: set[tuple[int,int]] = set()
beacons: set[tuple[int,int]] = set()
sb_pairs: list[tuple[tuple[int,int],tuple[int,int]]] = []

In [274]:
import math
x_min = math.inf
x_max = -math.inf
y_min = math.inf
y_max = -math.inf
largest_man = 0
non_beacon: set[tuple[int, int]] = set()

In [275]:
import re

regex = re.compile(r"Sensor at x=(\S+), y=(\S+): closest beacon is at x=(\S+), y=(\S+)")

matches = regex.findall(data)

for match in matches:
    int_conv = tuple(int(x) for x in match)
    sb_pairs.append(((int_conv[0],int_conv[1]),(int_conv[2],int_conv[3])))
    sensors.add((int_conv[0],int_conv[1]))
    beacons.add((int_conv[2],int_conv[3]))
    x_min = min(x_min, int_conv[0])
    x_max = max(x_max, int_conv[0])
    y_min = min(y_min, int_conv[1])
    y_max = max(y_max, int_conv[1])
    x_min = min(x_min, int_conv[2])
    x_max = max(x_max, int_conv[2])
    y_min = min(y_min, int_conv[3])
    y_max = max(y_max, int_conv[3])

In [276]:
def blast(coord: tuple[int,int], man_radius: int):
    global x_min, x_max, y_min, y_max
    cx, cy = coord
    for i in range(-man_radius, man_radius + 1):
        for j in range(-man_radius, man_radius + 1):
            if (abs(i) + abs(j)) <= man_radius and (cx+i, cy+j) not in beacons and (i, j) != (0,0):
                non_beacon.add((cx+i, cy+j))
                x_min = min(x_min, cx+i)
                x_max = max(x_max, cx+i)
                y_min = min(y_min, cy+i)
                y_max = max(y_max, cy+i)

In [277]:
sm_pairs: list[tuple[tuple[int,int],int]] = []
for sb_pair in sb_pairs:
    sensor, beacon = sb_pair
    manhattan = abs(beacon[0] - sensor[0]) + abs(beacon[1] - sensor[1])
    largest_man = max(manhattan, largest_man)
    sm_pairs.append((sensor, manhattan))

In [278]:
def test_in_splat(splat_rule: tuple[tuple[int,int],int], test_coord: tuple[int,int]):
    sensor_coord, man_dist = splat_rule
    return abs(test_coord[0] - sensor_coord[0]) + abs(test_coord[1] - sensor_coord[1]) <= man_dist

In [267]:
target_row = 2000000
not_contain_in_row = 0
for i in range(int(x_min)-largest_man-1,int(x_max) + largest_man + 2):
    for sm_pair in sm_pairs:
        in_splat = test_in_splat(sm_pair, (i, target_row))
        if in_splat and (i, target_row) not in beacons:
            not_contain_in_row += 1
            break

In [268]:
not_contain_in_row

0

## Part 2

In [269]:
import z3

In [279]:
def test_out_splat(splat_rule: tuple[tuple[int,int],int], test_coord: tuple[int,int]):
    sensor_coord, man_dist = splat_rule
    return z3.Abs(test_coord[0] - sensor_coord[0]) + z3.Abs(test_coord[1] - sensor_coord[1]) > man_dist

In [280]:
x, y = z3.Ints('x y')
constraints = []
for sm_pair in sm_pairs:
    sensor, manhattan = sm_pair
    constraints.append(test_out_splat(sm_pair, (x,y)))

sol = z3.solve([
    0 <= x, x <= 4000000,
    0 <= y, y <= 4000000] + constraints)

[x = 2895970, y = 2601918]


In [281]:
x = 2895970
y = 2601918
x * 4000000 + y

11583882601918